<a href="https://colab.research.google.com/github/Rosvend/Passenger-flow-prediction-LSTM/blob/main/LSTM_Metro_passenger_flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [84]:
from ipywidgets import interact, interact_manual

In [76]:
print('Descargando archivo...')
!curl -L "https://drive.google.com/uc?id=1rawKGyqGLCjhYsE1oePv499lQdawN3zX&export=download" -o Afluencia2023.xlsx

Descargando archivo...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  659k  100  659k    0     0   599k      0  0:00:01  0:00:01 --:--:-- 80.5M


In [77]:
df = pd.read_excel('/content/Afluencia2023.xlsx',engine='openpyxl')
df.head(13) #Imprimos las 12 lineas del metro

,Día,Línea de Servicio,Hora de operación,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,NaN,NaN,04:00:00,05:00:00,06:00:00,07:00:00,08:00:00,09:00:00,10:00:00,11:00:00,...,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00,Total general (Número de pasajeros)
1,01/01/2023,LÍNEA 1,192,811,896,716,673,922,1023,1208,...,1629,1985,2361,2694,2303,1839,1478,236,NaN,25483
2,01/01/2023,LÍNEA 2,11,123,116,95,102,144,147,189,...,191,255,280,317,252,164,111,7,NaN,3210
3,01/01/2023,LÍNEA A,1328,5104,5701,4309,4054,4852,5880,7629,...,10999,13027,15965,19123,15663,12918,9509,819,NaN,166047
4,01/01/2024,LÍNEA B,221,701,750,655,628,863,1085,1228,...,1930,2247,2498,2648,2029,1472,1087,151,NaN,25106
5,01/01/2023,LÍNEA H,NaN,NaN,NaN,NaN,NaN,56,62,49,...,55,128,99,109,68,46,27,1,NaN,970
6,01/01/2023,LÍNEA J,NaN,NaN,NaN,NaN,NaN,470,410,490,...,688,710,857,933,651,311,146,3,NaN,7209
7,01/01/2023,LÍNEA K,NaN,NaN,NaN,NaN,350,555,630,745,...,1432,1515,1788,1562,1110,766,329,4,NaN,13664
8,01/01/2023,LÍNEA L,NaN,NaN,NaN,NaN,11,61,158,364,...,617,531,428,NaN,NaN,NaN,NaN,NaN,NaN,4113
9,01/01/2023,LÍNEA M,NaN,NaN,NaN,NaN,1,156,123,176,...,229,332,319,411,408,260,240,92,NaN,3425


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4304 entries, 0 to 4303
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Día                4303 non-null   object
 1   Línea de Servicio  4303 non-null   object
 2   Hora de operación  3639 non-null   object
 3   Unnamed: 3         3644 non-null   object
 4   Unnamed: 4         3648 non-null   object
 5   Unnamed: 5         3647 non-null   object
 6   Unnamed: 6         4093 non-null   object
 7   Unnamed: 7         4291 non-null   object
 8   Unnamed: 8         4290 non-null   object
 9   Unnamed: 9         4290 non-null   object
 10  Unnamed: 10        4294 non-null   object
 11  Unnamed: 11        4295 non-null   object
 12  Unnamed: 12        4296 non-null   object
 13  Unnamed: 13        4291 non-null   object
 14  Unnamed: 14        4292 non-null   object
 15  Unnamed: 15        4297 non-null   object
 16  Unnamed: 16        4056 non-null   object


In [79]:
#Cambiamos los nombres de las columnas de horas
hour = 4
for i in range(2, 22):
    if hour < 12:
        new_column_name = f"{hour} AM"
    elif hour == 12:
        new_column_name = "12 PM"
    elif hour>12:
        new_column_name = f"{hour-12} PM"

    df = df.rename(columns={f"Unnamed: {i}": new_column_name})
    hour += 1

df.columns.to_list()

['Día',
 'Línea de Servicio',
 'Hora de operación',
 '5 AM',
 '6 AM',
 '7 AM',
 '8 AM',
 '9 AM',
 '10 AM',
 '11 AM',
 '12 PM',
 '1 PM',
 '2 PM',
 '3 PM',
 '4 PM',
 '5 PM',
 '6 PM',
 '7 PM',
 '8 PM',
 '9 PM',
 '10 PM',
 '11 PM',
 'Unnamed: 22']

In [80]:
df = df.rename(columns={'Hora de operación':'4 AM','Unnamed: 22':'Total_pasajeros'}) #Cambiamos el resto de columnas que faltaron
df = df.drop([0]) #Borramos la primera fila
df.head(12)

,Día,Línea de Servicio,4 AM,5 AM,6 AM,7 AM,8 AM,9 AM,10 AM,11 AM,...,3 PM,4 PM,5 PM,6 PM,7 PM,8 PM,9 PM,10 PM,11 PM,Total_pasajeros
1,01/01/2023,LÍNEA 1,192,811,896,716,673,922,1023,1208,...,1629,1985,2361,2694,2303,1839,1478,236,NaN,25483
2,01/01/2023,LÍNEA 2,11,123,116,95,102,144,147,189,...,191,255,280,317,252,164,111,7,NaN,3210
3,01/01/2023,LÍNEA A,1328,5104,5701,4309,4054,4852,5880,7629,...,10999,13027,15965,19123,15663,12918,9509,819,NaN,166047
4,01/01/2024,LÍNEA B,221,701,750,655,628,863,1085,1228,...,1930,2247,2498,2648,2029,1472,1087,151,NaN,25106
5,01/01/2023,LÍNEA H,NaN,NaN,NaN,NaN,NaN,56,62,49,...,55,128,99,109,68,46,27,1,NaN,970
6,01/01/2023,LÍNEA J,NaN,NaN,NaN,NaN,NaN,470,410,490,...,688,710,857,933,651,311,146,3,NaN,7209
7,01/01/2023,LÍNEA K,NaN,NaN,NaN,NaN,350,555,630,745,...,1432,1515,1788,1562,1110,766,329,4,NaN,13664
8,01/01/2023,LÍNEA L,NaN,NaN,NaN,NaN,11,61,158,364,...,617,531,428,NaN,NaN,NaN,NaN,NaN,NaN,4113
9,01/01/2023,LÍNEA M,NaN,NaN,NaN,NaN,1,156,123,176,...,229,332,319,411,408,260,240,92,NaN,3425
10,01/01/2023,LÍNEA O,10,122,172,99,107,138,95,150,...,216,234,272,326,222,198,109,14,NaN,2944


In [85]:
#Celda interactiva con hora y cantidad de pasajeros
@interact
def show_lines(column=['4 AM','5 AM','6 AM'	,'7 AM'	,'8 AM',	'9 AM',	'10 AM','11 AM','12 PM','1 PM', '2 PM','3 PM','4 PM',	'5 PM','6 PM','7 PM',	'8 PM',	'9 PM',	'10 PM','11 PM'],
                            x=(10, 10000, 10)):
    return df.loc[df[column] > x]

interactive(children=(Dropdown(description='column', options=('4 AM', '5 AM', '6 AM', '7 AM', '8 AM', '9 AM', …

In [82]:
#Web scraping del clima 2023
import requests
from bs4 import BeautifulSoup

file = requests.get('https://www.wunderground.com/history/daily/co/olaya/SKMD/date/2023-4-2')
soup = BeautifulSoup(file.content, "html.parser")

In [83]:
# create empty list
list =[]
all = soup.find("div", {"class":"locations-title ten-day-page-title"}).find("h1").text

# find all table with class-"twc-table"
content = soup.find_all("table", {"class":"twc-table"})
for items in content:
	for i in range(len(items.find_all("tr"))-1):
				# create empty dictionary
		dict = {}
		try:
						# assign value to given key

			dict["day"]= items.find_all("span", {"class":"date-time"})[i].text
			dict["date"]= items.find_all("span", {"class":"day-detail"})[i].text
			dict["desc"]= items.find_all("td", {"class":"description"})[i].text
			dict["temp"]= items.find_all("td", {"class":"temp"})[i].text
			dict["precip"]= items.find_all("td", {"class":"precip"})[i].text
			dict["wind"]= items.find_all("td", {"class":"wind"})[i].text
			dict["humidity"]= items.find_all("td", {"class":"humidity"})[i].text
		except:
					# assign None values if no items are there with specified class

			dict["day"]="None"
			dict["date"]="None"
			dict["desc"]="None"
			dict["temp"]="None"
			dict["precip"]="None"
			dict["wind"]="None"
			dict["humidity"]="None"

		# append dictionary values to the list
		list.append(dict)


AttributeError: 'NoneType' object has no attribute 'find'